# Transformers Pre-entrenados con HuggingFace

**Materiales desarrollados por Matías Barreto, 2025**

**Tecnicatura en Ciencia de Datos - IFTS**

**Asignatura:** Procesamiento de Lenguaje Natural

---

## Introducción

Este notebook marca un **punto de inflexión** en el curso. Hasta ahora construimos modelos desde cero (perceptrón, MLP, LSTM) para entender los fundamentos. Ahora damos el salto al **paradigma moderno de NLP**: usar modelos transformers pre-entrenados con transfer learning.

### El momento "ajá" del curso

Después de 5 notebooks programando y entrenando modelos, ahora vamos a ver que:
- **3 líneas de código** superan todo lo anterior
- No necesitamos entrenar desde cero
- Transfer learning es el estándar de la industria

Esto no invalida lo que aprendimos. Todo ese conocimiento te permite entender **qué está pasando bajo el capó** de HuggingFace.

### ¿Qué son los Transformers?

Arquitectura revolucionaria introducida en "Attention is All You Need" (Vaswani et al., 2017):
- **Attention mechanism**: Todas las palabras se ven entre sí simultáneamente
- **Paralelizable**: No procesa secuencialmente como LSTM
- **Escalable**: Funciona con modelos gigantes (GPT-3: 175B parámetros)
- **Transfer learning**: Pre-entrenamiento + fine-tuning

### Objetivos de aprendizaje

1. Usar HuggingFace Transformers library
2. Cargar modelos pre-entrenados (BETO, RoBERTuito)
3. Aplicar pipelines para tareas comunes
4. Entender el concepto de transfer learning
5. Trabajar con modelos en español
6. Prepararse para fine-tuning (próximas semanas del curso)

### ¿Por qué HuggingFace?

- **Model Hub**: Miles de modelos pre-entrenados
- **API unificada**: Mismo código para BERT, GPT, T5, etc.
- **Comunidad**: Open source, muy activa
- **Industria**: Estándar de facto en producción
- **Español**: Excelentes modelos para español (BETO, RoBERTuito, MarIA)

---

## 1. Instalación de HuggingFace Transformers

En Google Colab, instalamos la librería (ya viene en algunas versiones, pero mejor asegurarnos).

In [ ]:
# Instalamos transformers de HuggingFace
# -q: quiet mode (menos output)
!pip install -q transformers

print("Librería 'transformers' instalada correctamente.")

In [ ]:
# Importamos librerías
from transformers import pipeline
import warnings

# Suprimimos warnings menores para output más limpio
warnings.filterwarnings('ignore')

print("Librerías importadas.")
print("\nNota: La primera vez que uses un modelo, HuggingFace lo descargará.")
print("Esto puede tardar unos minutos dependiendo del tamaño del modelo.")

---

## 2. Concepto: Transfer Learning en NLP

### Paradigma tradicional (lo que hicimos en notebooks anteriores):

```
1. Recolectar dataset etiquetado
2. Inicializar pesos aleatoriamente
3. Entrenar desde cero
4. Evaluar
```

**Problemas:**
- Requiere muchos datos (>10k muestras)
- Tiempo de entrenamiento largo
- Recursos computacionales (GPU)
- El modelo aprende lenguaje desde cero

### Paradigma moderno: Transfer Learning

```
1. Tomar modelo pre-entrenado en corpus masivo (Wikipedia, Common Crawl)
2. Adaptar a tu tarea específica (fine-tuning)
3. Requiere pocos datos (<1k muestras)
4. Entrenamiento rápido
```

**Ventajas:**
- El modelo ya "entiende" lenguaje
- Solo ajusta para tu tarea específica
- Funciona con datasets pequeños
- Estado del arte con menos esfuerzo

### Analogía

**Entrenar desde cero:** Como enseñar a leer a alguien desde alfabeto y luego pedirle que clasifique sentimientos

**Transfer learning:** Como pedirle a alguien que ya lee español que aprenda a distinguir reseñas positivas/negativas (mucho más rápido)

### Dos fases del Transfer Learning

**Pre-entrenamiento (ya hecho por otros):**
- Corpus masivo: Wikipedia (3B palabras), Common Crawl (500B palabras)
- Tarea auto-supervisada: Masked Language Modeling (BERT), Next Token Prediction (GPT)
- Meses de entrenamiento en clusters de GPUs
- Costo: $100k - $1M USD

**Fine-tuning (lo que nosotros haremos):**
- Dataset específico: Nuestras reseñas etiquetadas
- Tarea supervisada: Clasificación, NER, QA, etc.
- Horas/días en una GPU
- Costo: $10 - $100 USD

---

## 3. HuggingFace Pipelines: La Forma Más Simple

Los **pipelines** son abstracciones de alto nivel que encapsulan:
1. Tokenización (texto → tokens)
2. Modelo (tokens → predicciones)
3. Post-procesamiento (predicciones → formato legible)

### Pipelines disponibles:

- `sentiment-analysis`: Clasificación de sentimientos
- `ner`: Named Entity Recognition
- `question-answering`: Responder preguntas sobre un contexto
- `text-generation`: Generar texto (GPT)
- `translation`: Traducción
- `summarization`: Resumen de texto
- `fill-mask`: Completar texto enmascarado
- Y muchos más...

### Sintaxis básica:

```python
# Opción 1: Modelo por defecto (inglés)
classifier = pipeline("sentiment-analysis")

# Opción 2: Modelo específico
classifier = pipeline("sentiment-analysis", model="finiteautomata/beto-sentiment-analysis")
```

---

## 4. Cargando BETO: BERT en Español

### ¿Qué es BETO?

**BETO** (Spanish BERT) es un modelo transformer pre-entrenado en español:
- Basado en BERT (Bidirectional Encoder Representations from Transformers)
- Pre-entrenado en Wikipedia en español
- 110M parámetros
- Desarrollado por Universidad de Chile

### Variante que usaremos:

`finiteautomata/beto-sentiment-analysis`:
- BETO fine-tuneado para análisis de sentimientos
- Entrenado en tweets en español
- 3 clases: POS (positivo), NEU (neutral), NEG (negativo)
- Estado del arte para español

In [ ]:
print("Cargando modelo BETO...")
print("La primera vez descargará ~400MB (modelo + tokenizer).")
print("Esto puede tardar 1-2 minutos.\n")

# Creamos el pipeline de sentiment analysis con BETO
# model: Especificamos el modelo del HuggingFace Model Hub
clasificador = pipeline(
    "sentiment-analysis",
    model="finiteautomata/beto-sentiment-analysis"
)

print("Modelo BETO cargado correctamente.")
print("\nInformación del modelo:")
print(f"  Nombre: finiteautomata/beto-sentiment-analysis")
print(f"  Base: BETO (Spanish BERT)")
print(f"  Tarea: Sentiment Analysis")
print(f"  Idioma: Español")
print(f"  Clases: POS, NEU, NEG")

---

## 5. Primera Predicción: ¡3 Líneas de Código!

Vamos a clasificar una frase. Observá lo simple que es comparado con todo lo que programamos antes.

In [ ]:
# Una sola frase de ejemplo
texto = "Si queres morirte de calor, el lugar está bárbaro, muy recomendable."

# Predicción (una línea)
resultado = clasificador(texto)

# Mostramos el resultado
print("="*70)
print("PREDICCIÓN CON TRANSFORMERS")
print("="*70)
print(f"\nTexto: '{texto}'")
print(f"\nResultado:")
print(f"  Sentimiento: {resultado[0]['label']}")
print(f"  Confianza: {resultado[0]['score']:.2%}")

print("\n" + "="*70)
print("Observación: ¡Esto fue mucho más simple que todos los notebooks")
print("anteriores juntos! El modelo ya viene entrenado y listo para usar.")

---

## 6. Corpus de Prueba: Español Rioplatense

Usamos el mismo corpus de los notebooks anteriores para comparar resultados.

In [ ]:
# Corpus en español rioplatense con expresiones coloquiales
frases = [
    # Positivas
    "La verdad, este lugar está bárbaro. Muy recomendable.",
    "Qué buena onda la atención, volvería sin dudarlo.",
    "Me encantó la comida, aunque la música estaba muy fuerte.",
    "Todo excelente. Atención de diez.",
    "Muy conforme con el resultado final.",
    "Superó mis expectativas, gracias.",
    "El mejor asado que probé en mucho tiempo.",
    "Excelente relación precio-calidad, muy recomendable.",
    "La atención fue impecable, muy atentos.",
    "Me gustó mucho el ambiente tranquilo.",

    # Negativas
    "Una porquería de servicio, nunca más vuelvo.",
    "El envío fue lento y el producto llegó dañado. Qué desastre.",
    "Qué estafa, me arrepiento de haber comprado.",
    "No me gustó para nada la experiencia.",
    "No lo recomiendo, mala calidad.",
    "Malísima atención, el mozo tenía mala onda.",
    "Tardaron dos horas en entregar, llegó todo frío.",
    "Me cobraron de más y encima se hicieron los giles.",
    "La carne estaba pasada, casi no se podía comer.",
    "Pésima experiencia, no vuelvo más.",

    # Adicionales con expresiones argentinas
    "Zafa, pero nada especial.",
    "Está piola el lugar, volvería.",
    "Qué garrón, tardaron una banda.",
    "Re copado todo, la rompieron.",
    "Un bodrio total, no vayan."
]

print(f"Corpus: {len(frases)} frases en español rioplatense")
print(f"\nEjemplos:")
print(f"  [Positiva] {frases[0]}")
print(f"  [Negativa] {frases[10]}")
print(f"  [Coloquial] {frases[20]}")

---

## 7. Clasificación Masiva con Pipelines

Los pipelines pueden procesar listas de textos de forma eficiente.

In [ ]:
print("Clasificando todas las frases con BETO...\n")

# Clasificamos todas las frases de una vez
# El pipeline automáticamente procesa en batches
resultados = clasificador(frases)

print("="*70)
print("RESULTADOS DE CLASIFICACIÓN")
print("="*70)

# Mostramos resultados detallados
for i, (frase, resultado) in enumerate(zip(frases, resultados), 1):
    label = resultado['label']
    score = resultado['score']

    # Truncamos frase si es muy larga
    frase_corta = frase if len(frase) <= 55 else frase[:52] + "..."

    # Formato de salida
    print(f"\n{i:2d}. '{frase_corta}'")
    print(f"    → {label} (confianza: {score:.2%})")

print("\n" + "="*70)

---

## 8. Análisis de Resultados

Analicemos qué tan bien BETO maneja expresiones coloquiales argentinas.

In [ ]:
# Extraemos las frases con expresiones muy argentinas
frases_argentinas = [
    ("Zafa, pero nada especial.", "NEU o POS"),  # Esperado
    ("Está piola el lugar, volvería.", "POS"),
    ("Qué garrón, tardaron una banda.", "NEG"),
    ("Re copado todo, la rompieron.", "POS"),
    ("Un bodrio total, no vayan.", "NEG"),
    ("Qué buena onda la atención.", "POS"),
    ("Malísima atención, mala onda.", "NEG"),
]

print("="*70)
print("ANÁLISIS: EXPRESIONES ARGENTINAS")
print("="*70)
print("\n¿BETO entiende jerga argentina?\n")

# Clasificamos estas frases
resultados_arg = clasificador([frase for frase, _ in frases_argentinas])

aciertos = 0
for (frase, esperado), resultado in zip(frases_argentinas, resultados_arg):
    predicho = resultado['label']
    score = resultado['score']

    # Verificamos si coincide
    correcto = predicho in esperado
    marca = "✓" if correcto else "?"
    if correcto:
        aciertos += 1

    print(f"{marca} '{frase}'")
    print(f"  Esperado: {esperado} | Predicho: {predicho} ({score:.2%})\n")

print("="*70)
print(f"Aciertos: {aciertos}/{len(frases_argentinas)}")
print("\nObservación: BETO fue entrenado en tweets en español, por lo que")
print("maneja bien expresiones coloquiales. Sin embargo, jerga muy específica")
print("de Argentina puede ser un desafío. Fine-tuning con datos locales mejoraría esto.")

---

## 9. Comparación con Modelos Anteriores

Comparemos conceptualmente lo que ganamos con transformers.

In [ ]:
print("="*70)
print("COMPARACIÓN: MODELOS CONSTRUIDOS VS. TRANSFORMERS")
print("="*70)

comparacion = [
    ("Líneas de código", "~200-300 líneas", "3 líneas"),
    ("Tiempo de desarrollo", "Horas/días", "Minutos"),
    ("Datos requeridos", ">1000 muestras", "0 (modelo ya entrenado)"),
    ("Tiempo de entrenamiento", "Minutos/horas", "0 (ya entrenado)"),
    ("Comprensión lingüística", "Limitada", "Profunda (pre-entrenado)"),
    ("Manejo de jerga", "Depende del vocabulario", "Robusto"),
    ("Contexto", "Limitado (BoW/LSTM)", "Bidireccional (BERT)"),
    ("Parámetros", "Decenas/cientos", "110 millones"),
    ("Accuracy esperado", "70-85%", "90-95%"),
    ("Fine-tuning", "N/A", "Posible con pocos datos"),
    ("Multilingüe", "Solo español", "Soporte nativo"),
    ("Interpretabilidad", "Alta (pesos directos)", "Media (attention)"),
    ("Recursos computacionales", "CPU suficiente", "GPU recomendada"),
    ("Tamaño del modelo", "KB", "400+ MB"),
]

print(f"\n{'Aspecto':<30} | {'Nuestros Modelos':<25} | {'Transformers (BETO)':<25}")
print("-"*85)
for aspecto, nuestros, transformers in comparacion:
    print(f"{aspecto:<30} | {nuestros:<25} | {transformers:<25}")

print("\n" + "="*70)
print("CONCLUSIÓN")
print("="*70)
print("\nLo que construimos en notebooks anteriores fue FUNDAMENTAL para:")
print("  1. Entender cómo funcionan las redes neuronales")
print("  2. Apreciar la complejidad que HuggingFace abstrae")
print("  3. Saber qué está pasando 'bajo el capó'")
print("  4. Debugging cuando algo falla")
print("\nPero para producción y proyectos reales:")
print("  → SIEMPRE usar transfer learning con modelos pre-entrenados")
print("  → HuggingFace es el estándar de la industria")
print("  → No reinventar la rueda (años-persona de investigación)")

---

## 10. Explorando Otros Modelos en Español

HuggingFace tiene múltiples modelos para español. Veamos algunos.

### 10.1. RoBERTuito: Especializado en Twitter Español

**RoBERTuito** es un modelo basado en RoBERTa entrenado en tweets en español:
- Mejor manejo de lenguaje coloquial
- Comprende jerga, emojis, hashtags
- Ideal para redes sociales

In [ ]:
print("Cargando RoBERTuito...\n")

# Cargamos pipeline con RoBERTuito
clasificador_twitter = pipeline(
    "sentiment-analysis",
    model="pysentimiento/robertuito-sentiment-analysis"
)

print("RoBERTuito cargado.")
print("\nProbando con frases estilo Twitter/redes sociales:\n")

# Frases tipo tweet
tweets = [
    "Jajaja re copado el lugar 😂👌",
    "Nooo qué garrón mal servicio 😤",
    "10/10 recomendadísimo 🔥",
    "Naaa una estafa total 👎👎",
]

resultados_twitter = clasificador_twitter(tweets)

for tweet, resultado in zip(tweets, resultados_twitter):
    print(f"Tweet: '{tweet}'")
    print(f"  → {resultado['label']} ({resultado['score']:.2%})\n")

### 10.2. Comparación BETO vs. RoBERTuito

Comparemos ambos modelos en las mismas frases.

In [ ]:
print("="*70)
print("COMPARACIÓN: BETO vs. RoBERTuito")
print("="*70)

frases_test = [
    "Está piola el lugar, re tranquilo.",
    "Qué bodrio, nunca más vuelvo.",
    "La comida estaba bien, nada del otro mundo.",
]

resultados_beto = clasificador(frases_test)
resultados_robertuito = clasificador_twitter(frases_test)

for i, frase in enumerate(frases_test):
    print(f"\nFrase: '{frase}'")
    print(f"  BETO:       {resultados_beto[i]['label']:8s} ({resultados_beto[i]['score']:.2%})")
    print(f"  RoBERTuito: {resultados_robertuito[i]['label']:8s} ({resultados_robertuito[i]['score']:.2%})")

print("\n" + "="*70)
print("Observaciones:")
print("  - BETO: Más formal, entrenado en Wikipedia")
print("  - RoBERTuito: Mejor con jerga y lenguaje informal")
print("  - Elección depende del dominio de tu aplicación")

---

## 11. Casos Difíciles: Límites de los Modelos

Incluso los transformers tienen límites. Veamos casos desafiantes.

In [ ]:
print("="*70)
print("CASOS DIFÍCILES: IRONÍA, SARCASMO, AMBIGÜEDAD")
print("="*70)

casos_dificiles = [
    # Ironía
    "Buenísimo, justo lo que necesitaba, tardar 3 horas.",

    # Sarcasmo
    "Claro, excelente idea cobrarme el doble.",

    # Sentimiento mixto
    "La comida excelente pero el servicio pésimo.",

    # Ambigüedad
    "Interesante experiencia.",

    # Negación doble
    "No estuvo nada mal.",
]

print("\nProbando BETO con casos desafiantes:\n")

resultados_dificiles = clasificador(casos_dificiles)

for caso, resultado in zip(casos_dificiles, resultados_dificiles):
    print(f"Frase: '{caso}'")
    print(f"  BETO dice: {resultado['label']} ({resultado['score']:.2%})")
    print(f"  Nota: ¿Capturó el matiz?\n")

print("="*70)
print("Observaciones:")
print("  - Ironía/sarcasmo: Muy difícil incluso para humanos sin contexto")
print("  - Sentimiento mixto: Los modelos devuelven una sola clase")
print("  - Ambigüedad: Score bajo indica incertidumbre del modelo")
print("\nSoluciones:")
print("  → Fine-tuning con datos específicos del dominio")
print("  → Modelos multiclase para sentimientos mixtos")
print("  → Features adicionales (emojis, contexto conversacional)")

---

## 12. Próximos Pasos: Fine-tuning

Lo que vimos en este notebook es **inferencia** (usar modelo ya entrenado). El siguiente paso es **fine-tuning**: adaptar el modelo a tu dataset específico.

### ¿Cuándo hacer fine-tuning?

**Usar modelo out-of-the-box:**
- Tarea general (sentiment analysis estándar)
- No tenés datos etiquetados
- Prototipado rápido
- Precisión suficiente (>85%)

**Hacer fine-tuning:**
- Dominio muy específico (medicina, legal, finanzas)
- Jerga particular (ej: argentinismos)
- Necesitás >90% accuracy
- Tenés dataset etiquetado (>500 muestras)

### Proceso de fine-tuning (próximas semanas del curso):

```python
from transformers import AutoModelForSequenceClassification, Trainer

# 1. Cargar modelo pre-entrenado
modelo = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

# 2. Preparar dataset
train_dataset = tokenize_and_encode(train_texts, train_labels)
eval_dataset = tokenize_and_encode(eval_texts, eval_labels)

# 3. Configurar entrenamiento
trainer = Trainer(
    model=modelo,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# 4. Fine-tune
trainer.train()
```

**Esto lo verán en detalle en las próximas semanas del curso.**

---

## Guía Teórico-Conceptual

### 1. Arquitectura Transformer: Conceptos Clave

**Attention Mechanism (Self-Attention):**

La innovación central de los transformers. Permite que cada palabra "preste atención" a todas las demás:

```
Frase: "El banco está cerrado"

Self-attention permite que "banco" mire:
  - "El" → Artículo (bajo peso)
  - "está" → Verbo estado (medio peso)
  - "cerrado" → ¡Alto peso! Disambigua "banco" (institución vs. asiento)
```

**Fórmula simplificada:**
$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

Donde:
- **Q (Query)**: "¿Qué busco?"
- **K (Key)**: "¿Qué ofrezco?"
- **V (Value)**: "¿Qué información tengo?"

**Multi-Head Attention:**
- Múltiples attention en paralelo
- Cada "cabeza" aprende patrones diferentes
- BERT tiene 12 capas × 12 cabezas = 144 atenciones

**Ventajas sobre LSTM:**
1. **Paralelización**: Todas las palabras se procesan simultáneamente
2. **Dependencias largas**: Cualquier palabra puede atender a cualquier otra
3. **Interpretabilidad**: Los pesos de attention son visualizables

### 2. BERT vs. GPT: Encoder vs. Decoder

| Aspecto | BERT | GPT |
|---------|------|-----|
| Arquitectura | Encoder (bidirectional) | Decoder (unidirectional) |
| Lectura | Lee toda la oración | Lee izquierda a derecha |
| Pre-entrenamiento | Masked LM | Next token prediction |
| Mejor para | Clasificación, NER, QA | Generación de texto |
| Ejemplos | BERT, RoBERTa, BETO | GPT-2, GPT-3, GPT-4 |

**Masked Language Modeling (BERT):**
```
Input:  "El [MASK] está cerrado"
Output: "banco" (con probabilidad 0.8)
```

**Next Token Prediction (GPT):**
```
Input:  "El banco está"
Output: "cerrado" (con probabilidad 0.6)
```

### 3. Tokenización en Transformers

Los transformers usan **sub-word tokenization** (WordPiece, BPE):

**Ventajas sobre tokenización por palabra:**
```
Palabra completa:
"recomendabilísimo" → <UNK> (desconocida)

WordPiece:
"recomendabilísimo" → ["recomienda", "##bili", "##simo"]
```

Esto permite:
- Vocabulario finito (30k tokens)
- Manejar palabras nuevas
- Capturar morfología (prefijos, sufijos)

### 4. Pre-entrenamiento: La Fase Cara

**Corpus de pre-entrenamiento:**
- BERT (inglés): Wikipedia (2.5B palabras) + BookCorpus (800M palabras)
- BETO (español): Wikipedia español (3B palabras)
- GPT-3: Common Crawl (570GB de texto)

**Recursos computacionales:**
- BERT: 4 días en 16 TPUs (~$7k)
- GPT-3: Semanas en clusters masivos (~$12M)

**Por esto NO entrenamos desde cero:**
- Costo prohibitivo
- Expertise técnico alto
- Modelos ya disponibles

### 5. Fine-tuning: La Fase Accesible

**Proceso:**
1. **Congelar capas inferiores**: Mantener conocimiento lingüístico general
2. **Entrenar capas superiores**: Adaptar a tarea específica
3. **Pocos epochs**: 2-5 típicamente
4. **Learning rate bajo**: 1e-5 a 5e-5

**Datos necesarios:**
- Clasificación: 500-1000 muestras
- NER: 1000-5000 entidades
- QA: 500-2000 pares pregunta-respuesta

**Tiempo:**
- GPU T4 (Google Colab): 30-60 minutos
- GPU A100: 5-10 minutos

### 6. HuggingFace Model Hub

**Estadísticas (2025):**
- 500,000+ modelos
- 100+ idiomas
- Todas las arquitecturas: BERT, GPT, T5, LLaMA, etc.

**Modelos destacados para español:**
1. **BETO** (`dccuchile/bert-base-spanish-wwm-cased`)
   - BERT base en español
   - 110M parámetros
   - General purpose

2. **RoBERTuito** (`pysentimiento/robertuito-*`)
   - RoBERTa en tweets español
   - Jerga, emojis, hashtags
   - Mejor para redes sociales

3. **MarIA** (`PlanTL-GOB-ES/roberta-base-bne`)
   - RoBERTa español
   - Corpus BNE (Biblioteca Nacional)
   - Muy robusto

4. **mBERT** (`bert-base-multilingual-cased`)
   - 104 idiomas incluyendo español
   - Útil para tareas multilingües

### 7. Limitaciones de Transformers

**1. Longitud de secuencia:**
- BERT: Máximo 512 tokens
- Documentos largos requieren truncamiento o chunking
- Soluciones: Longformer, BigBird (hasta 4096 tokens)

**2. Recursos computacionales:**
- Modelos grandes (400MB - 2GB)
- Inferencia más lenta que modelos simples
- GPU recomendada para fine-tuning

**3. Interpretabilidad:**
- 110M parámetros son difíciles de interpretar
- Attention weights ayudan pero no explican todo
- "Caja menos negra" que antes, pero aún opaco

**4. Sesgos del pre-entrenamiento:**
- Heredan sesgos del corpus (Wikipedia, internet)
- Requiere cuidado en aplicaciones sensibles
- Fine-tuning puede amplificar sesgos

**5. Overfitting en fine-tuning:**
- Con pocos datos, puede sobreajustar
- Requiere regularización, early stopping
- Data augmentation ayuda

---

## Preguntas y Respuestas para Estudio

### Preguntas Conceptuales

**1. ¿Qué es transfer learning en NLP y por qué es tan importante?**

*Respuesta:* Transfer learning es usar un modelo pre-entrenado en un corpus masivo (Wikipedia, Common Crawl) y adaptarlo a una tarea específica con pocos datos. Es importante porque:
1. El modelo ya "entiende" lenguaje (gramática, semántica)
2. Solo necesitás 500-1000 muestras en vez de millones
3. Ahorras meses de entrenamiento y miles de dólares
4. Obtenés estado del arte sin expertise en deep learning

**2. ¿Cuál es la principal innovación de los transformers sobre LSTM?**

*Respuesta:* **Self-attention mechanism**. En lugar de procesar secuencialmente (palabra por palabra), los transformers permiten que todas las palabras se "vean" entre sí simultáneamente. Esto:
- Permite paralelización (mucho más rápido)
- Captura dependencias largas mejor
- Es más interpretable (visualizar attention weights)

**3. ¿Qué es un pipeline en HuggingFace y qué problema resuelve?**

*Respuesta:* Un pipeline es una abstracción de alto nivel que encapsula:
1. **Tokenización**: Texto → tokens
2. **Modelo**: Tokens → embeddings → predicciones
3. **Post-procesamiento**: Predicciones → formato legible

Resuelve el problema de complejidad: en lugar de manejar manualmente tokenizers, modelos y configuraciones, el pipeline lo hace en una línea de código.

**4. ¿Por qué BERT es "bidirectional" y GPT es "unidirectional"?**

*Respuesta:*
- **BERT**: Lee toda la oración en ambas direcciones. En "El banco está cerrado", "banco" ve tanto "El" (izquierda) como "cerrado" (derecha)
- **GPT**: Solo lee de izquierda a derecha. En "El banco está", solo puede usar "El banco" para predecir "está"

Por eso BERT es mejor para clasificación/comprensión y GPT para generación.

**5. ¿Qué es Masked Language Modeling y por qué es efectivo para pre-entrenamiento?**

*Respuesta:* MLM oculta aleatoriamente el 15% de las palabras y el modelo debe predecirlas: "El [MASK] está cerrado" → "banco". Es efectivo porque:
- Fuerza al modelo a entender contexto bidireccional
- Es auto-supervisado (no requiere etiquetas humanas)
- Aprende representaciones profundas del lenguaje

### Preguntas Técnicas

**6. En el código, ¿qué hace exactamente `pipeline("sentiment-analysis", model="...")` internamente?**

*Respuesta:*
1. Descarga el modelo y tokenizer desde HuggingFace Hub (si no está en caché)
2. Carga el modelo pre-entrenado en memoria
3. Configura el tokenizer apropiado (WordPiece para BERT)
4. Crea un pipeline que encadena: tokenización → modelo → argmax → label

**7. ¿Por qué los modelos transformer son tan grandes (400MB+)?**

*Respuesta:*
- BERT base: 110M parámetros × 4 bytes (float32) = 440MB
- Cada parámetro es un peso de la red neuronal
- 12 capas × 12 attention heads × embeddings × FFN = muchos parámetros

Comparado con nuestro MLP (280 parámetros = 1KB), es ~400,000 veces más grande.

**8. ¿Qué significa el "score" en el resultado del pipeline?**

*Respuesta:* Es la probabilidad (softmax) que el modelo asigna a esa clase:
- score=0.95 → 95% seguro de la predicción
- score=0.55 → Apenas seguro (predicción incierta)

Útil para:
- Filtrar predicciones inciertas (threshold > 0.7)
- Priorizar casos para revisión humana

**9. ¿Cuál es la diferencia entre `model` y `pipeline` en HuggingFace?**

*Respuesta:*
- **Pipeline**: API de alto nivel, maneja todo automáticamente, fácil de usar
- **Model**: API de bajo nivel, control total, requiere manejar tokenización manualmente

```python
# Pipeline (simple)
resultado = pipeline("sentiment-analysis")("texto")

# Model (control manual)
tokenizer = AutoTokenizer.from_pretrained("beto")
model = AutoModel.from_pretrained("beto")
inputs = tokenizer("texto", return_tensors="pt")
outputs = model(**inputs)
```

**10. ¿Por qué BETO y RoBERTuito dan resultados diferentes en la misma frase?**

*Respuesta:*
1. **Corpus de pre-entrenamiento diferente**:
   - BETO: Wikipedia (formal)
   - RoBERTuito: Twitter (informal)
2. **Fine-tuning diferente**: Diferentes datos de sentiment analysis
3. **Vocabulario**: RoBERTuito conoce mejor jerga, emojis

La elección depende del dominio de tu aplicación.

### Preguntas de Aplicación

**11. Tenés 200 reseñas etiquetadas de un restaurante. ¿Usarías el modelo as-is o harías fine-tuning?**

*Respuesta:* **Depende:**
- **As-is si**: El modelo out-of-the-box ya da >85% accuracy, el dominio es general
- **Fine-tuning si**:
  - Necesitás >90% accuracy
  - Hay vocabulario muy específico del restaurante
  - 200 muestras son borderline (mínimo recomendado es 500), pero podés intentar con data augmentation

**12. ¿Cómo manejarías un documento de 2000 palabras con BERT (límite: 512 tokens)?**

*Respuesta:* Opciones:
1. **Truncamiento**: Tomar primeros 512 tokens (pierde info del final)
2. **Chunking + agregación**: Dividir en chunks de 512, clasificar cada uno, agregar (voting, promedio)
3. **Extractivo**: Identificar sección clave (ej: resumen, intro) y clasificar eso
4. **Modelo especializado**: Usar Longformer o BigBird (hasta 4096 tokens)

**13. En producción, necesitás clasificar 10,000 textos/segundo. ¿Transformers son apropiados?**

*Respuesta:* Probablemente **no** para latencia ultra-baja:
- BERT inference: ~50-100ms/texto en GPU
- 10k/seg = 0.1ms/texto (100x más rápido requerido)

**Soluciones:**
1. **Distillation**: DistilBERT (60% más rápido, 97% accuracy)
2. **Quantization**: INT8 en vez de FP32 (4x más rápido)
3. **Caching**: Cachear predicciones para textos comunes
4. **Ensemble**: Modelo simple (Naive Bayes) para mayoría, BERT solo para casos inciertos

**14. ¿Cómo usarías HuggingFace para un chatbot que genera respuestas?**

*Respuesta:*
```python
# Usar modelo generativo (GPT)
from transformers import pipeline

generador = pipeline("text-generation", model="gpt2")

respuesta = generador(
    "Usuario: Hola, ¿cómo estás?\nBot:",
    max_length=50,
    num_return_sequences=1
)
```

Para español, usar modelos como `DeepESP/gpt2-spanish`.

**15. Diseñá un sistema de moderación de contenido con transformers.**

*Respuesta:*
```python
# Pipeline multimodelo

# 1. Clasificación de toxicidad
toxicidad = pipeline("text-classification", model="modelo-toxicidad-es")

# 2. Detección de spam
spam = pipeline("text-classification", model="modelo-spam-es")

# 3. NER para detectar info personal
ner = pipeline("ner", model="modelo-ner-es")

def moderar(texto):
    # Análisis paralelo
    es_toxico = toxicidad(texto)[0]['score'] > 0.7
    es_spam = spam(texto)[0]['score'] > 0.8
    tiene_pii = any(ent['entity'] == 'PER' for ent in ner(texto))
    
    if es_toxico or es_spam:
        return "RECHAZAR"
    elif tiene_pii:
        return "ADVERTIR"
    else:
        return "APROBAR"
```

---

## Ejercicios Propuestos

### Ejercicio 1: Explorar el Model Hub
Visita https://huggingface.co/models y busca modelos para:
- Sentiment analysis en español
- NER en español
- Traducción es→en

Probá al menos 3 modelos diferentes y compará resultados.

### Ejercicio 2: Análisis de Confianza
Filtrá las predicciones del modelo para mostrar solo aquellas con score > 0.8. ¿Cuántas frases quedan? ¿Las predicciones de baja confianza tienen algo en común?

### Ejercicio 3: Comparación Cuantitativa
Evalúa BETO vs. RoBERTuito en un conjunto de 50 frases etiquetadas manualmente:
- Calcula accuracy, precision, recall
- Identifica en qué tipo de frases cada modelo es mejor

### Ejercicio 4: NER con Transformers
Usa un pipeline de NER para extraer entidades de texto:
```python
ner = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner")
```
Probá con textos sobre geografía, personas famosas, organizaciones.

### Ejercicio 5: Límites del Modelo
Crea 10 frases diseñadas para "engañar" al modelo (ironía, sarcasmo, negaciones dobles). ¿Qué porcentaje clasifica incorrectamente? ¿Ves patrones en los errores?

---

## Conclusión

Este notebook marca el **fin del módulo introductorio** y el **inicio del programa principal** del curso.

### Lo que aprendimos:

1. **Transfer learning**: El paradigma moderno de NLP
2. **HuggingFace Transformers**: La librería estándar de la industria
3. **Pipelines**: Abstracciones de alto nivel para inferencia
4. **Modelos en español**: BETO, RoBERTuito, MarIA
5. **Limitaciones**: Casos difíciles, sesgos, recursos

### El viaje hasta aquí:

```
Notebook 1-2: Baselines clásicos (sklearn, Naive Bayes)
     ↓
Notebook 3: Fundamentos (Perceptrón desde cero)
     ↓
Notebook 4: Redes profundas (MLP con PyTorch)
     ↓
Notebook 5: Secuencias (LSTM con Keras)
     ↓
Notebook 6: Estado del arte (Transformers con HuggingFace) ← ESTÁS AQUÍ
```

### ¿Qué sigue en el curso?

**Semanas 1-3: Fundamentos de transformers**
- Arquitectura en detalle
- Tokenización avanzada
- Preparación de datos

**Semanas 4-5: Fine-tuning**
- Clasificación binaria y multiclase
- Optimización de hiperparámetros
- Evaluación rigurosa

**Semana 6: NER y token classification**

**Semanas 7-8: Modelos generativos**
- GPT para generación
- Summarization
- Question Answering

**Semanas 9-11: Proyecto integrador**

### Reflexión final

Los 5 notebooks anteriores fueron **fundamentales** para entender cómo funcionan las redes neuronales. Ahora que vimos la potencia de HuggingFace, apreciás:

- El valor de abstracciones bien diseñadas
- Por qué transfer learning revolucionó NLP
- Que "hacer deep learning" hoy es usar herramientas correctamente, no programar todo desde cero

Pero también entendés **qué está pasando bajo el capó**, lo cual te diferencia de alguien que solo sabe usar pipelines sin comprensión profunda.

**Próximo paso:** El Notebook 7 (Embeddings Semánticos) es material complementario para proyectos avanzados. Luego continúa con el programa principal del curso.

---

*Este material fue desarrollado con fines educativos para la Tecnicatura en Ciencia de Datos del IFTS.*